# Language Translation

In this project, you’re going to take a peek into the realm of neural network machine translation. You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.

## Get the Data

Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function text_to_ids(), you'll turn source_text and target_text from words to ids. However, you need to add the <EOS> word id at the end of each sentence from target_text. This will help the neural network predict when the sentence should end.

You can get the <EOS> word id by doing:

    target_vocab_to_int['<EOS>']
You can get other word ids using source_vocab_to_int and target_vocab_to_int.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    
    source_sentences = [sentence for sentence in source_text.split('\n')]
    target_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_sentences]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_sentences]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed



### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

## Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.5.0


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:

model_inputs
* process_decoding_input
* encoding_layer
* decoding_layer_train
* decoding_layer_infer
* decoding_layer
* seq2seq_model

### Input
Implement the model_inputs() function to create TF Placeholders for the Neural Network. It should create the following placeholders:

* Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
* Targets placeholder with rank 2.
* Learning rate placeholder with rank 0.
* Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)



In [7]:
len(target_int_text)

137861

In [8]:
max_len_eng_sent = max([len(line) for line in source_int_text])
max_len_fra_sent = max([len(line) for line in target_int_text])

In [9]:
target_int_to_vocab = {v:k for k,v in target_vocab_to_int.items()}
source_int_to_vocab = {v:k for k,v in source_vocab_to_int.items()}

In [10]:
tokenized_eng_sentences = np.zeros(shape = (len(source_int_text),max_len_eng_sent,len(source_vocab_to_int)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (len(target_int_text),max_len_fra_sent,len(target_vocab_to_int)), dtype='float32')
target_data = np.zeros((len(target_int_text), max_len_fra_sent, len(target_vocab_to_int)),dtype='float32')

In [11]:
tokenized_eng_sentences.shape

(137861, 17, 231)

In [12]:
# Vectorize the english and french sentences

for i in range(len(target_int_text)):
    for k,ch in enumerate(source_int_text[i]):
        tokenized_eng_sentences[i,k,ch] = 1
        
    for k,ch in enumerate(target_int_text[i]):
        tokenized_fra_sentences[i,k,ch] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,ch] = 1

### Encoding
Implement encoding_layer() to create a Encoder RNN layer using tf.nn.dynamic_rnn().

In [14]:
# Encoder model

encoder_input = Input(shape=(None,len(source_vocab_to_int)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]


### Decoding - Training
Create training logits using tf.contrib.seq2seq.simple_decoder_fn_train() and tf.contrib.seq2seq.dynamic_rnn_decoder(). Apply the output_fn to the tf.contrib.seq2seq.dynamic_rnn_decoder() outputs.

In [15]:
# Decoder model

decoder_input = Input(shape=(None,len(target_vocab_to_int)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(target_vocab_to_int),activation='softmax')
decoder_out = decoder_dense (decoder_out)


In [16]:
decoder_out.shape


TensorShape([Dimension(None), Dimension(None), Dimension(358)])

In [17]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
          y=target_data,
          batch_size=64,
          epochs=50,
          validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/50
110288/110288 [==============================] - 289s 3ms/step - loss: 0.6684 - val_loss: 0.3432
Epoch 2/50
110288/110288 [==============================] - 126s 1ms/step - loss: 0.1828 - val_loss: 0.1247
Epoch 3/50
110288/110288 [==============================] - 125s 1ms/step - loss: 0.0432 - val_loss: 0.0295
Epoch 4/50
110288/110288 [==============================] - 123s 1ms/step - loss: 0.0243 - val_loss: 0.0274
Epoch 5/50
110288/110288 [==============================] - 118s 1ms/step - loss: 0.0173 - val_loss: 0.0227
Epoch 6/50
110288/110288 [==============================] - 117s 1ms/step - loss: 0.0137 - val_loss: 0.0145
Epoch 7/50
110288/110288 [==============================] - 117s 1ms/step - loss: 0.0113 - val_loss: 0.0134
Epoch 8/50
110288/110288 [==============================] - 117s 1ms/step - loss: 0.0097 - val_loss: 0.0119
Epoch 9/50
110288/110288 [==============================] - 117s 1ms/step - loss: 0.0

In [19]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [21]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(target_vocab_to_int)))
    #target_seq[0, 0, target_vocab_to_int[' ']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_fra_char = target_int_to_vocab[max_val_index]
        translated_sent += ' ' + sampled_fra_char
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(target_vocab_to_int)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent



In [22]:
for seq_index in range(10):
    inp_seq = tokenized_eng_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', [source_int_to_vocab[a] for a in source_int_text[seq_index]])
    print('Decoded sentence:', translated_sent)

-
Input sentence: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', ',', 'and', 'it', 'is', 'snowy', 'in', 'april', '.']
Decoded sentence:  est parfois calme à l'automne
-
Input sentence: ['the', 'united', 'states', 'is', 'usually', 'chilly', 'during', 'july', ',', 'and', 'it', 'is', 'usually', 'freezing', 'in', 'november', '.']
Decoded sentence:  est généralement froid en
-
Input sentence: ['california', 'is', 'usually', 'quiet', 'during', 'march', ',', 'and', 'it', 'is', 'usually', 'hot', 'in', 'june', '.']
Decoded sentence:  déteste les calme en mars
-
Input sentence: ['the', 'united', 'states', 'is', 'sometimes', 'mild', 'during', 'june', ',', 'and', 'it', 'is', 'cold', 'in', 'september', '.']
Decoded sentence:  est parfois légère en juin
-
Input sentence: ['your', 'least', 'liked', 'fruit', 'is', 'the', 'grape', ',', 'but', 'my', 'least', 'liked', 'is', 'the', 'apple', '.']
Decoded sentence:  moins aimé des fruits est
-
Input sentence: ['his', 'favorite', 'fruit', 